## Procesamiento de Lenguaje Natural

*MINI-TASK \#1* 

# ***Byte-Pair-Encoding del Quijote***

### **Equipo:**

- Giottonini Herrera Enrique Alejandro
- Burruel Durán Luis Andrés
- Jorge Andres Rascon Acuña
- Villalba Miranda Jesús Abraham

**Fuentes**
* [Byte Pair Encoding (Lei Mao) ](https://leimao.github.io/blog/Byte-Pair-Encoding/)
* [Byte-Pair Encoding: Subword-based tokenization algorithm (Chetna Khanna) ](https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0)
---

## **Introducción**

El **byte pair encoding (BPE)** es una técnica sencilla de comprimir datos que consiste en reemplazar la combinación de bytes más frecuente en los datos con un byte que no aparece en ellos.

El objetivo es que al terminar de procesar los datos las palabras más comunes estén representadas en el vocabulario como un solo token, mientras que las palabras raras se dividen en dos o más tokens de subpalabras, lo cual coincide con lo que hace un algoritmo de **subword-based tokenization**.

Los datos que se utilizaran para esta tarea sera _"Don Quijote de la Mancha"_ de _Miguel de Cervantes Saavedra_ en version EBOOK transcrito por _The Project Gutenberg_.

## Aprendiendo de nuestro dataset
Antes de armar el algoritmo de **BPE** primero explicaremos las funciones que necesitamos.

Importamos las librerias que necesitamos

In [1]:
import re, collections

### **Vocab per words**
Explicación de la función

La función get_vocab se encarga de obtener las palabras contenidas en el vocabulario del corpus y la frecuencia de estas. Para esto, la función recibe el parámetro filename, esto es, el archivo a analizar. 

In [2]:
def get_vocab(filename):
    vocab = collections.defaultdict(int) #Define primero un diccionario vacío llamado "vocab"
    #A continuación, se abre un archivo con la función open, donde la regresa como archivo objeto.
    #Se utiliza como parámetros 'filename', 'r' que indica que se va a leer el archivo (read) y 
    #Por otro lado, utf-8 es un formato de codificación de caracteres Unicode e ISO 10646.
    with open(filename, 'r', encoding='utf-8') as fhand:  
        for line in fhand: #lee cada linea (line)
            words = line.strip().split() #se quitan espacios en blanco y se separan palabras

            #Finalmente, incluye cada palabra en el diccionario y su frecuencia, anexando al final </w>
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1 
    return vocab #Regresa el diccionario "vocab" con el vocabulario

In [3]:
vocabulario = get_vocab('corpus/extract.txt')

In [4]:
print("Vocabulario inicial:")
for word, freq in vocabulario.items():
    print(f"{word}: {freq}")

Vocabulario inicial:
E l </w>: 3
i n g e n i o s o </w>: 2
h i d a l g o </w>: 2
d o n </w>: 1
Q u i j o t e </w>: 1
d e </w>: 19
l a </w>: 6
M a n c h a </w>: 1
T A S A </w>: 1
Y o , </w>: 1
J u a n </w>: 2
G a l l o </w>: 2
A n d r a d a , </w>: 1
e s c r i b a n o </w>: 1
C á m a r a </w>: 1
d e l </w>: 4
R e y </w>: 1
n u e s t r o </w>: 1
s e ñ o r , </w>: 1
l o s </w>: 3
q u e </w>: 7
r e s i d e n </w>: 1
e n </w>: 6
s u </w>: 2
C o n s e j o , </w>: 1
c e r t i f i c o </w>: 1
y </w>: 10
d o y </w>: 1
f e </w>: 1
q u e , </w>: 1
h a b i e n d o </w>: 1
v i s t o </w>: 1
p o r </w>: 2
s e ñ o r e s </w>: 1
d é l </w>: 1
u n </w>: 1
l i b r o </w>: 4
i n t i t u l a d o </w>: 1
M a n c h a , </w>: 1
c o m p u e s t o </w>: 1
M i g u e l </w>: 1
C e r v a n t e s </w>: 1
S a a v e d r a , </w>: 1
t a s a r o n </w>: 1
c a d a </w>: 1
p l i e g o </w>: 1
d i c h o </w>: 4
a </w>: 4
t r e s </w>: 2
m a r a v e d í s </w>: 2
m e d i o ; </w>: 1
e l </w>: 3
c u a l </w>: 1
t i e n e <

### **Obteniendo los pares**

La función get_stats tiene como parámetro el diccionario vocab generado con la función get_vocab, y devuelve otro diccionario "pairs" con la frecuencia de los pares de tokens más recurrentes.

In [5]:
def get_stats(vocab):
    pairs = collections.defaultdict(int) #Define primero un diccionario vacío llamado "pairs"
    for word, freq in vocab.items():
        symbols = word.split() #Separa la palabra en tokens
        for i in range(len(symbols)-1): #Recorre todos los tokens de la palabra, excluyendo </w>
            #Se suman las veces que se repite una palabra a la frecuencia del par de tokens
            pairs[symbols[i],symbols[i+1]] += freq 
    return pairs #Regresa el diccionario con los pares de tokens mas recurrentes y su frecuencia

In [6]:
pairs = get_stats(vocabulario)

In [7]:
sorted_pairs = sorted(get_stats(vocabulario).items(), key=lambda x:x[1],reverse=True)
for i in range(10):
    print(sorted_pairs[i])

(('e', '</w>'), 41)
(('o', '</w>'), 36)
(('a', '</w>'), 30)
(('d', 'e'), 29)
(('e', 'n'), 22)
(('l', '</w>'), 16)
(('n', '</w>'), 16)
((',', '</w>'), 16)
(('e', 's'), 15)
(('s', '</w>'), 15)


In [8]:
best = max(pairs, key=pairs.get)
print(f"El par de caracteres consecutivos más frecuentes es: {best}")

El par de caracteres consecutivos más frecuentes es: ('e', '</w>')


### **Merge**
La función `merge_vocab` es la encargada de actualizar el vocabulario al mezclar el par de caracteres más frecuente. 

Esta función recibe:
1. **pair**: par más frecuente en el vocabulario
2. **v_in**: el vocabulario.

La función regresa el vocabulario actualizado despues de unir los caracteres y actualizar la frequencia de los tokens.

In [9]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' ' .join(pair)) # ("w1", "w2") -> "w1 w2"
    # Expresión regular utilizada para buscar el par más frecuente de caracteres
    # en el vocabucario"
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

Para probar la función utilizamos el par más frecuente de caracteres de `vocabulario`

In [10]:
new_vocab = merge_vocab(best, vocabulario)
print(f"El par mas frequente: {best}")
print("Nuevo vocabulario:\n")
for word, freq in new_vocab.items():
    print(f"{word}: {freq}")

El par mas frequente: ('e', '</w>')
Nuevo vocabulario:

E l </w>: 3
i n g e n i o s o </w>: 2
h i d a l g o </w>: 2
d o n </w>: 1
Q u i j o t e</w>: 1
d e</w>: 19
l a </w>: 6
M a n c h a </w>: 1
T A S A </w>: 1
Y o , </w>: 1
J u a n </w>: 2
G a l l o </w>: 2
A n d r a d a , </w>: 1
e s c r i b a n o </w>: 1
C á m a r a </w>: 1
d e l </w>: 4
R e y </w>: 1
n u e s t r o </w>: 1
s e ñ o r , </w>: 1
l o s </w>: 3
q u e</w>: 7
r e s i d e n </w>: 1
e n </w>: 6
s u </w>: 2
C o n s e j o , </w>: 1
c e r t i f i c o </w>: 1
y </w>: 10
d o y </w>: 1
f e</w>: 1
q u e , </w>: 1
h a b i e n d o </w>: 1
v i s t o </w>: 1
p o r </w>: 2
s e ñ o r e s </w>: 1
d é l </w>: 1
u n </w>: 1
l i b r o </w>: 4
i n t i t u l a d o </w>: 1
M a n c h a , </w>: 1
c o m p u e s t o </w>: 1
M i g u e l </w>: 1
C e r v a n t e s </w>: 1
S a a v e d r a , </w>: 1
t a s a r o n </w>: 1
c a d a </w>: 1
p l i e g o </w>: 1
d i c h o </w>: 4
a </w>: 4
t r e s </w>: 2
m a r a v e d í s </w>: 2
m e d i o ; </w>: 1
e l </w>

### **Get tokens**
La función `get_tokens` regresa los los tipos y su frecuencia del vocabulario del corpus.

In [11]:
def get_tokens(vocab):
    tokens = collections.defaultdict(int)
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens[token] += freq
    return tokens

Los tipos de nuestro vocabulario serian

In [12]:
tokens = get_tokens(vocabulario)

Ahora elaboramos una función auxiliar para guardar los tipos y su frecuencia en un archivo de texto

In [13]:
def save_vocab(tokens, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"Tipo, Frecuencia\n")
        for token, freq in tokens.items():
            file.write(f"{token}, {freq}\n")

In [14]:
save_vocab(tokens, 'vocab.txt')

### **Armando el algoritmo**

In [22]:
def byte_pair_encoding(vocab, iter):
    vocab_aux = vocab
    for i in range(iter):
        pairs = get_stats(vocab_aux)
        if not pairs:
            break
        print(f"Iter {i+1}:")
        best = max(pairs, key=pairs.get)
        vocab_aux = merge_vocab(best, vocab_aux)
        tokens = get_tokens(vocab_aux)
        print(f"Mejor par {best} \nNúmero de tokens: {len(tokens)}")
        print('============')
    return vocab_aux

## **Aplicandolo al Quijote**

In [16]:
vocabulario = get_vocab('corpus/quijote.txt')

In [17]:
save_vocab(get_tokens(vocabulario), 'vocabulario_inicial.txt')

**Número de iteraciones del algoritmo:**

In [18]:
num_merges = 1000

Utilizando el algoritmo

In [33]:
vocabulario = byte_pair_encoding(vocabulario,num_merges)

Iter 1:
Mejor par ('tras', 'lu') 
Número de tokens: 7010
Iter 2:
Mejor par ('dilig', 'encia,</w>') 
Número de tokens: 7011
Iter 3:
Mejor par ('r', 'o.</w>') 
Número de tokens: 7012
Iter 4:
Mejor par ('Señ', 'or,</w>') 
Número de tokens: 7013
Iter 5:
Mejor par ('pobr', 'es,</w>') 
Número de tokens: 7014
Iter 6:
Mejor par ('menos', 'cabo</w>') 
Número de tokens: 7015
Iter 7:
Mejor par ('quej', 'as</w>') 
Número de tokens: 7016
Iter 8:
Mejor par ('al', 'ti') 
Número de tokens: 7017
Iter 9:
Mejor par ('solici', 't') 
Número de tokens: 7018
Iter 10:
Mejor par ('favor', 'eci') 
Número de tokens: 7019
Iter 11:
Mejor par ('lic', 'os</w>') 
Número de tokens: 7020
Iter 12:
Mejor par ('a', 'go</w>') 
Número de tokens: 7021
Iter 13:
Mejor par ('peligr', 'osa</w>') 
Número de tokens: 7022
Iter 14:
Mejor par ('condici', 'ón,</w>') 
Número de tokens: 7023
Iter 15:
Mejor par ('toc', 'antes</w>') 
Número de tokens: 7024
Iter 16:
Mejor par ('en', 'comi') 
Número de tokens: 7025
Iter 17:
Mejor par ('carr

**Guardamos nuestro vocabulario**

In [34]:
save_vocab(get_tokens(vocabulario), 'vocab_quijote_8000x.txt')

## **Encoding and Decoding**

## **Conclusiones**